# Generalizing the Majorana Representation for Mixed States and Operators

So one of the biggest limitations of the Majorana representation for spin is that it only works for pure states. But what happens if we want to deal with mixed states (or operators more generally)? Is there some natural generalization, also in terms of constellations that transform nicely under rotations?

Leboeuf in his "Phase space approach to quantum dynamics," suggests one avenue. We know that we can write the Majorana polynomial for a single spin as $f(z) = \langle \tilde{z} \mid \psi \rangle$ where $\tilde{z}$ is the point antipodal to $z$ on the sphere. For two spins, we could consider the two variable polynomial:

$f(z_{0}, z_{1}) = \langle \tilde{z_{0}} \tilde{z_{1}} \mid \psi \rangle = \sum_{m_{0} = -j_{0}}^{j_0} \sum_{m_{1} = -j_{1}}^{j_{1}} c_{m_{0}, m_{1}}z_{0}^{j_{0}-m_{0}}z_{1}^{j_{1}-m_{1}}$

And then consider "cross-sections" like:

$s_{m_{1}}(z_{0}) = \sum_{m_{0}=-j_{0}}^{j_{0}} c_{m_{0}, m_{1}}z_{0}^{j_{0}-m_{0}}$

for each value of $m_{1}$. There will be $2j_{1} + 1$ such functions and each will have $2j_{0}$ zeros. This set of $(2j_{1} + 1) \times 2j_{0}$ zeros completely determines the quantum state of the two spins. In other words, given two spins $A$ and $B$, we can consider each of the $m$ values of $B$, and get a set of constellations describing $A$, one for each $m$ value of $B$ (and vice versa).

<hr>

But that's not entirely satisfying, and we can do better. In what follows, we rely on the very nice recent paper "Majorana representation for mixed states," where they employ the "Ramachandran-Ravishankar representation" aka the "T-rep".

The basic tool is a set of tensor operators for a given $j$ value: $\{T_{\sigma, \mu}^{j}\}_{\mu=-\sigma}^{\sigma}$ : these are sets of matrices which transform nicely under $SU(2)$, and are the matrix analogue of the spherical harmonics $Y_{lm}(\theta, \phi)$ which span the space of real valued functions on the sphere.

They are defined by:

$T_{\sigma, \mu}^{j} = \sum_{m, m^{\prime}=-j}^{j} (-1)^{j-m^\prime} C_{j, m; j, -m'}^{\sigma, \mu} \mid j, m \rangle \langle j, m^{\prime} \mid$

For a given value of $j$, $\sigma$ ranges from $0$ to $2j$, and $\mu$ from $-\sigma$ to $\sigma$. The $C$ refers to the Clebsch-Gordan coefficient $C_{j_{1}, m_{1}; j_{2}, m_{2}}^{j, m}$.

Recall the latter's meaning in terms of angular momentum coupling theory. We have:

$\mid j, m \rangle = \sum_{m_{1}=-j_{1}}^{j_{1}} \sum_{m_{2}=-j_{2}}^{j_{2}} \mid j_{1}, m_{1}; j_{2}, m_{2} \rangle \langle j_{1}, m_{1};j_{2}, m_{2} \mid j, m \rangle$

Then: $C_{j_{1}, m_{1}; j_{2}, m_{2}}^{j, m} = \langle j_{1}, m_{1};j_{2}, m_{2} \mid j, m \rangle$.

We're imagining that we have the tensor product of two spins with $j_{1}$ and $j_{2}$, and we want to switch to the "coupled basis" (in terms of eigenstates of the total angular momentum): the Clebsch-Gordan coefficients tell us how to do that.

In any case, for a given $j$ value, we can construct a collection of "spherical tensors". There will be $2j$ sets of them (for $\sigma$ from $0$ to $2j)$, each with $2\sigma+1$ operators in a set. 

In [1]:
from spheres import *

def spherical_tensor(j, sigma, mu):
    terms = []
    for m1 in np.arange(-j, j+1):
        for m2 in np.arange(-j, j+1):
            terms.append(\
                ((-1)**(j-m2))*\
                qt.clebsch(j, j, sigma, m1, -m2, mu)*\
                qt.spin_state(j, m1)*qt.spin_state(j, m2).dag())
    return sum(terms)

def spherical_tensor_basis(j):
    T_basis = {}
    for sigma in np.arange(0, int(2*j+1)):
        for mu in np.arange(-sigma, sigma+1):
            T_basis[(sigma, mu)] = spherical_tensor(j, sigma, mu)
    return T_basis

j = 3/2
T = spherical_tensor_basis(j)
for sigma_mu, O in T.items():
    print("sigma: %d, mu: %d\n%s\n" % (sigma_mu[0], sigma_mu[1], O))

<IPython.core.display.Javascript object>

sigma: 0, mu: 0
Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0.5 0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  0.5]]

sigma: 1, mu: -1
Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[0.         0.         0.         0.        ]
 [0.54772256 0.         0.         0.        ]
 [0.         0.63245553 0.         0.        ]
 [0.         0.         0.54772256 0.        ]]

sigma: 1, mu: 0
Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 0.67082039  0.          0.          0.        ]
 [ 0.          0.2236068   0.          0.        ]
 [ 0.          0.         -0.2236068   0.        ]
 [ 0.          0.          0.         -0.67082039]]

sigma: 1, mu: 1
Quantum object: dims = [[4], [4]], shape = (4, 4), type = oper, isherm = False
Qobj data =
[[ 0.         -0.54772256  0.          0.        ]
 [ 0.          0.         -0.63245553  0.       

For a given $j$ value, corresponding to a $2j+1$ dimensional representation, these matrices span the set of all $(2j+1) \times (2j+1)$ complex square matrices. And so, we decompose any operator (like a Hermitian density matrix) in terms of them. The coefficients in this basis will be generally complex.

In [2]:
def operator_spherical_decomposition(O, T_basis=None):
    j = (O.shape[0]-1)/2
    if not T_basis:
        T_basis = spherical_tensor_basis(j)
    decomposition = {}
    for sigma in np.arange(0, int(2*j+1)):
        for mu in np.arange(-sigma, sigma+1):
            decomposition[(sigma, mu)] = (O*T_basis[(sigma, mu)].dag()).tr()
    return decomposition

O = qt.rand_dm(int(2*j+1))
coeffs = operator_spherical_decomposition(O, T_basis=T)
for sigma_mu, c in coeffs.items():
    print("sigma: %d, mu: %d = %.3f + %.3f" % (sigma_mu[0], sigma_mu[1], c.real, c.imag))

sigma: 0, mu: 0 = 0.500 + 0.000
sigma: 1, mu: -1 = 0.057 + -0.020
sigma: 1, mu: 0 = -0.163 + 0.000
sigma: 1, mu: 1 = -0.057 + -0.020
sigma: 2, mu: -2 = 0.029 + -0.049
sigma: 2, mu: -1 = -0.025 + 0.095
sigma: 2, mu: 0 = 0.027 + 0.000
sigma: 2, mu: 1 = 0.025 + 0.095
sigma: 2, mu: 2 = 0.029 + 0.049
sigma: 3, mu: -3 = 0.020 + -0.085
sigma: 3, mu: -2 = 0.023 + 0.038
sigma: 3, mu: -1 = 0.029 + 0.023
sigma: 3, mu: 0 = 0.052 + 0.000
sigma: 3, mu: 1 = -0.029 + 0.023
sigma: 3, mu: 2 = 0.023 + -0.038
sigma: 3, mu: 3 = -0.020 + -0.085


Naturally, we can go in reverse and recover the operator in the standard basis.

In [3]:
def spherical_decomposition_operator(decomposition, T_basis=None):
    j = max([k[0] for k in decomposition.keys()])/2
    if not T_basis:
        T_basis = spherical_tensor_basis(j)
    terms = []
    for sigma in np.arange(0, int(2*j+1)):
        for mu in np.arange(-sigma, sigma+1):
            terms.append(decomposition[(sigma, mu)]*T_basis[(sigma, mu)])
    return sum(terms)

O2 = spherical_decomposition_operator(coeffs, T_basis=T)
print("recovered operator? %s" % np.allclose(O,O2))

recovered operator? True


Looking at the decomposition, you might observe that it looks like a tower of spin states with integer values for $j$! We have complex coefficients for $\mid 0, 0\rangle$, complex coefficients for $\mid 1, -1 \rangle$, $\mid 1, 0 \rangle$, $\mid 1, 1 \rangle$, and so forth. 

In other words, we could think about this decomposition as giving us a set of spins. Notice however that the spins states are *not* normalized.

In [4]:
def spherical_decomposition_spins(decomposition):
    max_j = max([k[0] for k in decomposition.keys()])
    return [qt.Qobj(np.array([decomposition[(j, m)]\
                        for m in np.arange(j, -j-1, -1)]))\
                            for j in np.arange(0, max_j+1)]

spins = spherical_decomposition_spins(coeffs)

for spin in spins:
    print(spin)
    print("norm: %.3f\n" % spin.norm())

Quantum object: dims = [[1], [1]], shape = (1, 1), type = bra
Qobj data =
[[0.5]]
norm: 0.500

Quantum object: dims = [[3], [1]], shape = (3, 1), type = ket
Qobj data =
[[-0.05735872-0.01979219j]
 [-0.16317337+0.j        ]
 [ 0.05735872-0.01979219j]]
norm: 0.184

Quantum object: dims = [[5], [1]], shape = (5, 1), type = ket
Qobj data =
[[ 0.02861079+0.04901568j]
 [ 0.02459963+0.09513075j]
 [ 0.02720236+0.j        ]
 [-0.02459963+0.09513075j]
 [ 0.02861079-0.04901568j]]
norm: 0.163

Quantum object: dims = [[7], [1]], shape = (7, 1), type = ket
Qobj data =
[[-0.02039833-0.08492803j]
 [ 0.02258204-0.03844422j]
 [-0.0293161 +0.02329892j]
 [ 0.05201113+0.j        ]
 [ 0.0293161 +0.02329892j]
 [ 0.02258204+0.03844422j]
 [ 0.02039833-0.08492803j]]
norm: 0.157



Naturally, each one of these spins states can be associated to a constellation, and in fact, under rotations of the operator, each constellation in this decomposition rotates as expected. Insofar as the norms are not 1, we could imagine that the radius of each constellation's sphere is given by the norm. Moreover, in the normal Majorana representation, the phase is thrown out: here, however, the relative phases between each constellation does matter, and has to be kept track of.

Now let's look at what these constellations are actually like:

In [5]:
for spin in spins:
    print(spin_xyz(spin))
    print()

[[0 0 0]]

[[ 0.43999211 -0.15182358 -0.88507432]
 [-0.43999211  0.15182358  0.88507432]]

[[ 0.46757668  0.16799182 -0.86784261]
 [ 0.18106385 -0.97598578 -0.12111003]
 [-0.18106385  0.97598578  0.12111003]
 [-0.46757668 -0.16799182  0.86784261]]

[[ 0.57930327  0.42994694 -0.69249791]
 [-0.91340961  0.27945732 -0.29595015]
 [-0.10296128 -0.9690529  -0.22435563]
 [ 0.10296128  0.9690529   0.22435563]
 [ 0.91340961 -0.27945732  0.29595015]
 [-0.57930327 -0.42994694  0.69249791]]



We note the important fact that: every star has an antipodal twin on the other side of the sphere. In other words, the stars come in opposite pairs. This is true for any Hermitian matrix. For a unitary matrix, however, this won't be true.

In [6]:
U = qt.rand_unitary(int(2*j+1))
for spin in operator_spins(U, T_basis=T):
    print(spin_xyz(spin))
    print()

[[0 0 0]]

[[-0.43499016  0.5684879  -0.69828724]
 [ 0.61934811 -0.4581386   0.63758681]]

[[-0.09655261 -0.66193919 -0.74331292]
 [-0.28787102  0.88181202 -0.37354764]
 [ 0.24550167 -0.75846402  0.60370627]
 [ 0.47873896  0.60746561  0.63387265]]

[[-0.24693462 -0.82824452 -0.50302516]
 [ 0.05194309  0.89419808 -0.44464785]
 [-0.93706059  0.34912572  0.00535633]
 [ 0.24735871  0.91366575  0.32253459]
 [ 0.86531585 -0.49235821  0.09387153]
 [-0.15893154 -0.83783322  0.52227987]]



Let's consider a pure state density matrix and compare it to the usual Majorana representation of the pure state.

In [7]:
pure_state = qt.rand_ket(int(2*j+1))
pure_dm = pure_state*pure_state.dag()

print("pure state stars:")
print(spin_xyz(pure_state))

print("\npure dm stars:")
for spin in operator_spins(pure_dm, T_basis=T):
    print(spin_xyz(spin))
    print()

pure state stars:
[[-0.68817937 -0.23027966 -0.68802648]
 [ 0.0909218   0.98752801 -0.12853656]
 [ 0.25267089 -0.51724818  0.81768682]]

pure dm stars:
[[0 0 0]]

[[ 0.84670229 -0.4948951  -0.19538186]
 [-0.84670229  0.4948951   0.19538186]]

[[ 0.50741376  0.04097585 -0.86072775]
 [ 0.85778717 -0.51326187 -0.02763006]
 [-0.85778717  0.51326187  0.02763006]
 [-0.50741376 -0.04097585  0.86072775]]

[[-0.25267089  0.51724818 -0.81768682]
 [-0.68817937 -0.23027966 -0.68802648]
 [ 0.0909218   0.98752801 -0.12853656]
 [-0.0909218  -0.98752801  0.12853656]
 [ 0.68817937  0.23027966  0.68802648]
 [ 0.25267089 -0.51724818  0.81768682]]



Considering the final constellation in the list, we can see that indeed this spherical tensor representation naturally generalizes the Majorana representation. The final constellation in the list with 6 stars corresponds to the 3 stars in the original pure state Majorana constellation, along with the three points opposite to them. In fact, one could ditch half the stars and still have a complete representation (as long as you remember to add them back in). The authors propose doing precisely this as well as a nice, but slightly involved scheme for fixing the phases, which we shall leave to the side.

And what significance we can ascribe to the other constellations in the decomposition, we shall see!

But first, let's do some visualization.

In [10]:
from spheres import *
scene = vp.canvas(background=vp.color.white)

j = 3/2
spin = qt.rand_ket(int(2*j+1))
dm = spin*spin.dag()

msphere = MajoranaSphere(spin, radius=1/2, position=vp.vector(-1,0,0), scene=scene)
osphere = OperatorSphere(dm, pos=vp.vector(1,0,0), scene=scene)

H = qt.jmat(j, 'y')
U = (-1j*H*0.01).expm()

for t in range(1000):
    spin = U*spin
    dm = U*dm*U.dag()
    msphere.spin = spin
    osphere.set(dm)
    vp.rate(1000)

<IPython.core.display.Javascript object>

`OperatorSphere` allows one to visualize a mixed state or operator using the spherical tensor decomposition. You see a series of concentric spheres whose radii are the norms of the spins, and the colors of the spheres/stars correspond to the *phase* of the spin states. You can see that the constellation with the highest $j$ value contains the original Majorana constellation (shown on the left), but also includes the antipodal points. And everything transforms nicely under and $SU(2)$ rotation. (And the label gives the value of the spin-$0$ sector).

We can look at the evolution of a density matrix under some random Hamiltonian. Notice that unlike in the case of simple rotations, the radii of the spheres can change--but the antipodal symmetry is preserved. (It's also interesting to think about states which have antipodal symmetry as being "time reversal invariant," insofar as the operator that inverts the spheres is for spins the time-reversal operator.)

In [22]:
from spheres import *
scene = vp.canvas(background=vp.color.white)

j = 3/2
dm = qt.rand_dm(int(2*j+1))
osphere = OperatorSphere(dm, scene=scene)

H = qt.rand_herm(int(2*j+1))
U = (-1j*H*0.01).expm()
osphere.evolve(H, dt=0.01, T=1000)

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

We can also evolve a random unitary operator. Notice that the antipodal symmetry is broken.

In [12]:
from spheres import *
scene = vp.canvas(background=vp.color.white)

j = 3/2
U = qt.rand_unitary(int(2*j+1))
osphere = OperatorSphere(U, scene=scene)

H = qt.rand_herm(int(2*j+1))
U = (-1j*H*0.01).expm()
osphere.evolve(H, dt=0.01, T=1000)

<IPython.core.display.Javascript object>

KeyboardInterrupt: 

Now here's an interesting thing. Let's take a spin state, convert it into a permutation symmetric multiqubit state, and look at the partial traces. In other words, we can look at the partial state of a single one of these qubits, of two of these qubits, three, etc. It doesn't matter which qubits we choose to look at because of the permutation symmetry, and moreover the partial states are themselves permutation symmetric, so we can convert those multiqubit density matrices into spin density matrices.

In [18]:
from spheres import *
scene = vp.canvas(background=vp.color.white)

j = 3/2
spin = qt.rand_ket(int(2*j+1))
sym = spin_sym(spin)

o = OperatorSphere(spin*spin.dag(), pos=vp.vector(-1,0,0), scene=scene)

for i in range(1, int(2*j)):
    partial = sym.ptrace(range(i))
    sym_map = spin_sym_map(i/2)
    OperatorSphere(sym_map.dag()*partial*sym_map, pos=vp.vector(2*j-i,0,0), scene=scene)

<IPython.core.display.Javascript object>

If it isn't visually obvious, let's compare the coordinates of the constellations in each of the partial traces with the coordinates of constellations in the overall state:

In [19]:
for i in range(1, int(2*j)):
    print("ptrace %s" % list(range(i)))
    partial = sym.ptrace(range(i))
    sym_map = spin_sym_map(i/2)
    partial_spin = sym_map.dag()*partial*sym_map
    for s in operator_spins(partial_spin):
        print("%s\n" % str(spin_xyz(s)))
        
print("full state:")
for s in operator_spins(spin*spin.dag()):
    print(spin_xyz(s))
    print()

ptrace [0]
[[0 0 0]]

[[ 0.19008415  0.91445326 -0.35727196]
 [-0.19008415 -0.91445326  0.35727196]]

ptrace [0, 1]
[[0 0 0]]

[[ 0.19008415  0.91445326 -0.35727196]
 [-0.19008415 -0.91445326  0.35727196]]

[[ 0.17670785 -0.44259581 -0.87913781]
 [ 0.7129492   0.68977917 -0.12612745]
 [-0.7129492  -0.68977917  0.12612745]
 [-0.17670785  0.44259581  0.87913781]]

full state:
[[0 0 0]]

[[ 0.19008415  0.91445326 -0.35727196]
 [-0.19008415 -0.91445326  0.35727196]]

[[ 0.17670785 -0.44259581 -0.87913781]
 [ 0.7129492   0.68977917 -0.12612745]
 [-0.7129492  -0.68977917  0.12612745]
 [-0.17670785  0.44259581  0.87913781]]

[[ 0.55856659  0.21995978 -0.79976313]
 [-0.24191822  0.69825361 -0.67373397]
 [ 0.91948558 -0.09641603 -0.38111706]
 [ 0.24191822 -0.69825361  0.67373397]
 [-0.91948558  0.09641603  0.38111706]
 [-0.55856659 -0.21995978  0.79976313]]



In other words, we see a very interesting feature of this representation: the partial states of the permutation symmetric qubits are already hidden inside the representation of the overall state! We get them all at once, "for free."

And indeed, this allows us to read off entanglement-related information from the spherical representation itself. For example, for three qubits, there are ultimately two different entanglement classes: GHZ-style entanglement and W-style entanglement. States within these classes can be transformed into each other by local quantum operations, but local operations can't convert between the two classes.

The GHZ state is: $\frac{1}{\sqrt{2}}(\mid \uparrow \uparrow \uparrow \rangle + \mid \downarrow \downarrow \downarrow \rangle)$. Crucially, if we measure one of the subsystems , for example, determining it to be $\uparrow$ or $\downarrow$, the other two subsystems will be steered to $\mid \uparrow \uparrow \rangle$ or $\mid \downarrow \downarrow \rangle$, which are separable states.

Now the GHZ state is permutation symmetric, and so we can convert it into a spin-$\frac{3}{2}$ state. Let's visualize it and its partial traces.

In [20]:
scene = vp.canvas(background=vp.color.white)
GHZsym = (bitstring_basis("111") + bitstring_basis("000"))/np.sqrt(2)
GHZspin = sym_spin(GHZsym)

OperatorSphere(GHZspin*GHZspin.dag(), pos=vp.vector(-1,0,0))
for i in range(1, 3):
    partial = GHZsym.ptrace(range(i))
    sym_map = spin_sym_map(i/2)
    OperatorSphere(sym_map.dag()*partial*sym_map, pos=vp.vector(2*j-i,0,0), scene=scene)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The single qubit partial state is maximally mixed, and the two qubit partial trace has a single sphere. 

On the other hand, we can consider the $W$ state: $\frac{1}{\sqrt{3}}(\mid \uparrow \uparrow \downarrow \rangle + \mid \uparrow \downarrow \uparrow \rangle + \mid \downarrow \uparrow \uparrow \rangle)$. In terms of entanglement, unlike the GHZ case, if one of the three qubits is lost, then the remaining two qubits will still be entangled. 

The $W$ state is also permutation symmetric, and so we can visualize it similarly:

In [21]:
scene = vp.canvas(background=vp.color.white)
Wsym = (bitstring_basis("100") + bitstring_basis("010") + bitstring_basis("001"))/np.sqrt(3)
Wspin = sym_spin(Wsym)

OperatorSphere(Wspin*Wspin.dag(), pos=vp.vector(-1,0,0))
for i in range(1, 3):
    partial = Wsym.ptrace(range(i))
    sym_map = spin_sym_map(i/2)
    OperatorSphere(sym_map.dag()*partial*sym_map, pos=vp.vector(2*j-i,0,0), scene=scene)

<IPython.core.display.Javascript object>

We can see that the single qubit partial state is *not* maximally mixed, and that the two qubit partial state has two spheres. And from this geometrical difference, we can read off the different entanglement properties of the $GHZ$ state and the $W$ state: if we lose a qubit of the $GHZ$ state, the remaining two qubits are not entangled; but if we lose a qubit of the $W$ state, the remaining two qubits will be.

<hr>

This turns out to be the tip of a very interesting iceberg, the details of which we'll have to save for another time, and I refer you again to the excellent paper "Majorana representation for mixed states." In short, a) in the pure state case, one can relate tensor contraction on the multiqubit side to taking *derivatives* of the Majorana polynomial b) one can describe a mixed state (or operator) in terms of a polynomial in four variables, so that partial tracing can be related to taking derivatives with respect to these variables c) formulate things like the multiplication of operators and the evaluation of expectation values in terms of differentiation--indeed, in the latter case, differentiation with respect to the stars! 

And more!

## Bibliography

<a href="https://iopscience.iop.org/article/10.1088/0305-4470/24/19/021">Phase space approach to quantum dynamics</a>

<a href="https://arxiv.org/abs/1909.07740">Majorana representation for mixed states</a>

<a href="https://arxiv.org/abs/1308.3777">Local Unitary Equivalent Classes of Symmetric N-Qubit Mixed States</a>

<a href="https://arxiv.org/abs/1706.01198">Geometric Multiaxial Representation of N-qubit Mixed Symmetric Separable States</a>